In [1]:
import numpy as np
import pandas as pd 
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import PolynomialFeatures
from sklearn import linear_model
import matplotlib.pyplot as plt 
%matplotlib notebook 
import os
import utils
reload(utils)

<module 'utils' from 'utils.pyc'>

In [2]:
X,supply,demand=utils.LoadDataset('./training_set/DesignMatrices/')
X=X.drop(['Temperature','PM2.5'],axis=1) #Dropping other two features for now 
Xnp,supplynp,demandnp=X.values,supply.values,demand.values

In [3]:
Xtrain, supplyTrain, demandTrain, Xval, supplyVal, demandVal=utils.SplitByRegion(Xnp,supplynp,demandnp,.7)


In [4]:
print Xtrain.shape, supplyTrain.shape, demandTrain.shape 
print Xval.shape, supplyVal.shape,demandVal.shape 

(99981L, 3L) (99981L,) (99981L,)
(42899L, 3L) (42899L,) (42899L,)


In [5]:
def one_hot_encode(x, n_classes):
    """
    One hot encode a list of sample labels. Return a one-hot encoded vector for each label.
    : x: List of sample Labels
    : return: Numpy array of one-hot encoded labels
     """
    return np.eye(n_classes)[x]

In [6]:
np.isnan(Xnp[:,2]).sum()

0

In [7]:
regID_ohe = one_hot_encode(Xtrain[:,0],n_classes=67)[:,1:]

In [8]:
weatherID_ohe = one_hot_encode(Xtrain[:,2],10)[:,1:]
weatherID_ohe

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  1., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  1., ...,  0.,  0.,  0.],
       [ 0.,  1.,  0., ...,  0.,  0.,  0.]])

In [9]:
def myTransform(X, d):
    """One-hot-encodes region number and weather, while raises timeslot to degree d"""
    a = one_hot_encode(X[:,0],n_classes=67)[:,1:]
    c = one_hot_encode(X[:,2],n_classes=10)[:,1:]
    b = X[:,1][:, np.newaxis]
    b = PolynomialFeatures(degree=d).fit_transform(b)
    
    out=np.concatenate((a, b, c), axis=1)
    return out 

In [10]:
bestscore = np.inf 
bestmodel = None
bestdeg = None
for d in np.arange(1,10+1):
    Xd = myTransform(Xtrain, d)
    model=linear_model.RidgeCV(alphas=np.logspace(1,20,5),)
    model.fit(Xd, demandTrain-supplyTrain)
    
    Xd_ = myTransform(Xval, d)
    ypreds = model.predict(Xd_)
    ypreds2 = model.predict(Xd)
    
    score = utils.MeanAbsoluteError(ypreds,demandVal-supplyVal)
    score2 = utils.MeanAbsoluteError(ypreds2,demandTrain-supplyTrain)
    if bestscore > score: 
        bestscore = score 
        bestmodel = model 
        bestdeg = d
    print 'degree = {}, score = {}, score2 = {}'.format(d, score, score2)

degree = 1, score = 10.7523501303, score2 = 10.7111965451
degree = 2, score = 11.0813239474, score2 = 11.0410598427
degree = 3, score = 284.328341849, score2 = 285.082318582
degree = 4, score = 7863501.4205, score2 = 7884879.73189
degree = 5, score = 1.26470215261e+11, score2 = 1.2689547166e+11
degree = 6, score = 1.87912088503e+15, score2 = 1.88640458331e+15
degree = 7, score = 2.19925240518e+19, score2 = 2.20816071115e+19
degree = 8, score = 6.05968394822e+23, score2 = 6.08320121343e+23
degree = 9, score = 1.2875881378e+23, score2 = 1.29199425008e+23
degree = 10, score = 5.00225838723e+22, score2 = 5.01735387898e+22


In [14]:
fnames = [x for x in sorted(os.listdir('./test_set/DesignMatrices/')) if x[0]!='.']

for fname in fnames: 
    Xtestpd = pd.read_csv('./test_set/DesignMatrices/'+fname)
    Xtestpd = Xtestpd[Xtestpd.Timeslot <= 51]
    
    Xtest = Xtestpd[['RegionID','Timeslot', 'Weather']].values
    print np.unique(Xtest[:,1])
    #Xtest = myTransform(Xtest.astype(int), bestdeg)
    
    #ypreds = bestmodel.predict(Xtest)
    
    #utils.WriteOnKaggleFormat(Xtestpd, ypreds, fname.split('.')[0])

[ 43.  44.  45.]
[]
[ 43.  44.  45.]
[]
[ 43.  44.  45.]


In [63]:
Xtestpd.columns

Index([u'Unnamed: 0', u'Timeslot', u'RegionHash', u'Date', u'RegionID',
       u'Weather', u'Temperature', u'PM2.5'],
      dtype='object')